In [42]:
import pandas as pd
import json
import numpy as np
from google.cloud import bigquery

In [43]:
#conversion du fichier json en json line delimiter pour utiliser pd.read_json avec une chunk size. (Nombre de ligne processée en une fois)
#!cat ../raw_data/export_23-5_minified.json | jq -c '.[]' > ../raw_data/export_converted_2.json

In [44]:
#Nombre de lignes totales dans le JSON : 1320229

In [45]:
#Localisation du projet name et table_id sur gcp
project="marbotic"
table_id = "marbotic_dataset.events"

In [46]:
#intégration des credentials 
from google.oauth2 import service_account

key_path = "/Users/antonin/code/AntoninAnq/gcp/marbotic-7d02fac30bd8.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [60]:
CHUNKSIZE = 100000

file_path = '../raw_data/export_converted_2.json'
df = pd.read_json(file_path, lines=True, chunksize=CHUNKSIZE) #Chunk size pour avoir 14 chunk

client = bigquery.Client(project,credentials)

for i,c in enumerate(df):
    index_init = np.linspace(1,len(c),len(c),dtype='int32')
    ep=pd.DataFrame()
    c.reset_index(inplace=True)
    ep=c.copy()
    ep.drop(['user_properties','event_properties'],axis=1,inplace=True)
    ep['user_id'] = ep['user_id'].apply(lambda x: 0 if (x is None or type(x)== str or  np.isnan(x)) else int(x))
    ep['id'] = (i*CHUNKSIZE) + index_init
    ep.drop(['index'],axis=1,inplace=True)
    #ep.to_csv('../raw_data/events.csv',index=False)
    job = client.load_table_from_dataframe(ep, table_id)  # Make an API request. ,job_config=job_config
    job.result()  # Wait for the job to complete.
    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )

Loaded 100000 rows and 15 columns to marbotic_dataset.events
Loaded 200000 rows and 15 columns to marbotic_dataset.events
Loaded 300000 rows and 15 columns to marbotic_dataset.events
Loaded 400000 rows and 15 columns to marbotic_dataset.events
Loaded 500000 rows and 15 columns to marbotic_dataset.events
Loaded 600000 rows and 15 columns to marbotic_dataset.events
Loaded 700000 rows and 15 columns to marbotic_dataset.events
Loaded 800000 rows and 15 columns to marbotic_dataset.events
Loaded 900000 rows and 15 columns to marbotic_dataset.events
Loaded 1000000 rows and 15 columns to marbotic_dataset.events
Loaded 1100000 rows and 15 columns to marbotic_dataset.events
Loaded 1200000 rows and 15 columns to marbotic_dataset.events
Loaded 1300000 rows and 15 columns to marbotic_dataset.events
Loaded 1320229 rows and 15 columns to marbotic_dataset.events


In [52]:
#data = pd.read_csv('../raw_data/events_test.csv')
#data
